# Mathematical Model for Acoustic Replication of Traditional Sasando

## 0. Abstract

This document presents a mathematical model for replicating the acoustic characteristics of the traditional sasando instrument in electric versions. The model employs a dual Fourier series representation to capture both the harmonic structure of the string vibration and the frequency-dependent amplification effects of the resonator. The approach extracts parameters from reference acoustic signals and applies them to transform electric sasando input signals, preserving the instrument's characteristic timbre while maintaining the electric instrument's pitch and timing.


## 1. Introduction

The traditional sasando is an Indonesian string instrument where sound is produced by vibrating strings that are acoustically amplified through a hand-formed, dried lontar leaf resonator. This unique resonator naturally amplifies frequencies between 98 Hz and 1047 Hz, creating the instrument's characteristic acoustic timbre. Modern electric versions replace the acoustic resonator with a pickup and speaker system, sacrificing acoustic quality for portability.

This work presents a mathematical model that enables electric sasando signals to replicate the acoustic characteristics of the traditional instrument. The model decomposes the acoustic signal into its harmonic components and models the resonator's contribution as a frequency-dependent transformation, providing a foundation for real-time sound synthesis in electric instruments.

## 2. Mathematical Model

### 2.1 Model Formulation

The complete model for replicating traditional sasando acoustic characteristics is:

$$
f(t) = A_0 e^{-t/\tau} \cdot \left[ \sum_{n=0}^{N} A_n \cos\left(n\omega_0 t + \phi_n\right) + \sum_{n=0}^{N} \alpha_n A_n \cos\left(n\omega_0 t + \phi_n + \theta\right) \right]
$$

where $N = 399$ (400 harmonics: $n = 0$ to $399$). The model consists of three main components:

1. **Harmonic Decomposition**: Signal represented as sum of harmonics with amplitudes $A_n$ and phases $\phi_n$
2. **Resonator Effect**: Second series adds frequency-dependent amplification ($\alpha_n$) and phase shift ($\theta$)
3. **Temporal Decay**: Exponential envelope ($A_0 e^{-t/\tau}$) captures natural decay of plucked strings

### 2.2 Model Components

**First Series:** $\sum_{n=0}^{N} A_n \cos(n\omega_0 t + \phi_n)$
- Represents the original signal decomposed into harmonics
- $A_n$: Amplitude coefficient for the $n$-th harmonic (extracted from input signal)
- $\phi_n$: Phase coefficient for the $n$-th harmonic (extracted from input signal)
- $\omega_0 = 2\pi f_0$: Fundamental angular frequency (rad/s)
- $n$: Harmonic index (0 = DC, 1 = fundamental, 2, 3, ..., 399)

**Second Series:** $\sum_{n=0}^{N} \alpha_n A_n \cos(n\omega_0 t + \phi_n + \theta)$
- Represents the resonator's contribution to the sound
- $\alpha_n$: Frequency-dependent amplitude scaling factor (models resonator response)
- $\theta$: Phase shift introduced by the resonator (constant, $\pi/4$ rad)

**Exponential Decay Envelope:** $E(t) = A_0 e^{-t/\tau}$
- $A_0$: Initial amplitude (extracted from the signal's peak envelope)
- $\tau$: Decay time constant (fitted from the signal's envelope using exponential regression)
- The envelope is extracted using the Hilbert transform

### 2.3 Resonator Response Function

The function $\alpha_n$ models the frequency-dependent amplification of the sasando resonator:

$$
\alpha_n = \begin{cases}
    0.8 & \text{if } 98 \text{ Hz} \leq n \cdot f_0 \leq 1047 \text{ Hz} \\
    0.2 & \text{otherwise}
\end{cases}
$$

## 3. Parameter Extraction

### 3.1 A$_n$ Extraction Formula

When extracting the amplitude coefficients $A_n$ from a recorded signal (which already includes resonator effects), we must account for the resonator's contribution. The FFT magnitude $M_n$ at frequency $n\omega_0$ represents the combined magnitude of both series terms.

#### 3.1.1 Derivation

Starting with the sum of two cosine terms:
$$
s(t) = A_n \cos(n\omega_0 t + \phi_n) + \alpha_n A_n \cos(n\omega_0 t + \phi_n + \theta)
$$

Using the trigonometric identity $\cos(a + b) = \cos(a)\cos(b) - \sin(a)\sin(b)$:

$$
s(t) = A_n [1 + \alpha_n \cos(\theta)] \cos(n\omega_0 t + \phi_n) - \alpha_n A_n \sin(\theta) \sin(n\omega_0 t + \phi_n)
$$

This can be written as a single sinusoid $R \cos(n\omega_0 t + \phi_n + \delta)$, where the magnitude $R$ is:

$$
R = A_n \sqrt{[1 + \alpha_n \cos(\theta)]^2 + [\alpha_n \sin(\theta)]^2} = A_n \sqrt{1 + 2\alpha_n \cos(\theta) + \alpha_n^2}
$$

Since $M_n$ (from FFT) represents the magnitude $R$ of the combined signal, we solve for $A_n$:

$$
A_n = \frac{M_n}{\sqrt{1 + 2\alpha_n \cos(\theta) + \alpha_n^2}}
$$



#### 3.1.2 Physical Interpretation

- $M_n$: The measured magnitude from FFT (includes both direct signal and resonator contribution)
- $A_n$: The base amplitude before resonator effect is applied
- The denominator $\sqrt{1 + 2\alpha_n \cos(\theta) + \alpha_n^2}$ accounts for how the resonator amplifies and phase-shifts the signal
- When $\alpha_n = 0$, $A_n = M_n$
- When $\theta = 0$, $A_n = M_n / (1 + \alpha_n)$

## 4. Model Assumptions

The mathematical model is based on the following assumptions:

1. **Harmonic Structure**: The signal can be represented as a sum of harmonics (integer multiples of the fundamental frequency). Inharmonic partials are negligible or approximated by nearby harmonics.

2. **Linear Resonator Model**: The resonator's effect is modeled as a linear transformation with frequency-dependent amplitude scaling $\alpha_n$ and constant phase shift $\theta$. The resonator behaves linearly without significant non-linear distortion.

3. **Exponential Decay**: The amplitude envelope follows an exponential decay model $A_0 e^{-t/\tau}$. The decay is uniform across all harmonics and the decay time constant $\tau$ is constant throughout the signal.

4. **Stationary Signal**: The signal characteristics (fundamental frequency, harmonic amplitudes, phases) are stationary over the analysis window, except for the exponential decay envelope.

5. **Perfect Harmonic Alignment**: All frequency components are exact integer multiples of the fundamental frequency $f_0$.

6. **Noise-Free Extraction**: Noise and continuous spectral content do not significantly affect the extraction of harmonic amplitudes and phases.

7. **Resonator Frequency Response**: The resonator's frequency response is approximated by a step function:
   $$
   \alpha_n = \begin{cases}
       0.8 & \text{if } 98 \text{ Hz} \leq n \cdot f_0 \leq 1047 \text{ Hz} \\
       0.2 & \text{otherwise}
   \end{cases}
   $$

8. **Constant Phase Shift**: The phase shift $\theta$ introduced by the resonator is constant across all frequencies.



## 5. References

- Oppenheim, A. V., & Schafer, R. W. (2010). *Discrete-Time Signal Processing* (3rd ed.). Prentice Hall. Section 2.6: Sinusoidal Signals and Phasors.
- Proakis, J. G., & Manolakis, D. G. (2007). *Digital Signal Processing* (4th ed.). Prentice Hall. Chapter 2: Discrete-Time Signals and Systems.
